In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
def extract_seconds_from_launch(game_state_str):
    try:
        return json.loads(game_state_str).get('seconds_from_launch', 0)
    except json.JSONDecodeError:
        return 0

In [3]:
df = pd.read_csv(r'C:\Users\oguo2\GitHub\PenguinsAI\RawData\PENGUINS_20240101_to_20240131_df72162_events.tsv', delimiter='\t')

In [4]:
def session_then_launch(target_csv, output_name, output_status):
    target_csv['seconds_from_launch'] = target_csv['game_state'].apply(extract_seconds_from_launch)
    df_sorted = target_csv.sort_values(by=['session_id', 'index'])
    if output_status:
        df_sorted.to_csv(output_name, sep='\t', index=False)
    else:
        return df_sorted


In [12]:
df_sorted = session_then_launch(df,"Sorted_Jan", True)

In [6]:
df.columns

Index(['session_id', 'app_id', 'timestamp', 'event_name', 'event_data',
       'event_source', 'app_version', 'app_branch', 'log_version', 'offset',
       'user_id', 'user_data', 'game_state', 'index', 'seconds_from_launch'],
      dtype='object')

In [7]:
def check_event_starts(group, event_name):
    mismatches = []
    # Get indices of the specified event
    event_indices = group[group['event_name'] == event_name].index
    # Check each event occurrence
    for idx in event_indices:
        # If it's not the first event or if the previous 'session_id' is the same, it's a mismatch
        if idx != group.index[0] and group.at[idx, 'session_id'] == group.at[idx - 1, 'session_id']:
            mismatches.append(group.at[idx, 'session_id'])
    return mismatches

In [8]:
def find_mismatches(df, event_name):
    # Group by 'session_id' and apply the checking function for the specified event name
    mismatched_sessions = df.groupby('session_id').apply(lambda g: check_event_starts(g, event_name))

    # Flatten the list of mismatched sessions
    mismatched_sessions = [item for sublist in mismatched_sessions for item in sublist]

    # Print out the sessions with mismatches
    if mismatched_sessions:
        print(f"The total of '{len(mismatched_sessions)}'following sessions have '{event_name}' events that do not match a change in 'session_id':")
        for session in mismatched_sessions:
            print(session)
    else:
        print(f"All '{event_name}' events match a change in 'session_id'.")


In [9]:
find_mismatches(df_sorted, 'device_identifier')

All 'device_identifier' events match a change in 'session_id'.


C:\Users\oguo2\AppData\Local\Temp\ipykernel_1836\3782325686.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  mismatched_sessions = df.groupby('session_id').apply(lambda g: check_event_starts(g, event_name))


In [10]:
len(df[df['index']==0])

471

In [20]:
df['event_name'].unique()

array(['global_timer_begin', 'mating_dance_indicator_updated', 'start',
       'device_identifier', 'begin', 'close_menu', 'application_start',
       'right_hand_data', 'gaze_object_begin', 'player_waddle',
       'viewport_data', 'left_hand_data', 'gaze_object_end',
       'activity_begin', 'pickup_rock', 'activity_end', 'nest_complete',
       'egg_hatch_indicator_updated', 'skua_spawn', 'skua_move',
       'flipper_bash_skua', 'egg_recovered', 'open_menu', 'egg_lost',
       'select_menu_item', 'headset_off', 'headset_on', 'egg_hatched',
       'language_selected', 'ring_chime'], dtype=object)